# 07 - Making and using hostart files

## Using PAVICS-Hydro to create a hotstart file to resume a simulation from given hydrological conditions

Here we run a hydrological model with `Raven` over a long period. Then we run the model on half of the period, save the final states, and then run the second half in an independent run but using the model states at the end of the first half. This should provide two (2) hydrographs that are exactly equal.

In [ ]:
import datetime as dt
import xarray as xr

At this point the following block of code should be quite familiar! If not, please go back to notebook "04 - Emulating hydrological models" to understand what is happening.

In [ ]:
# Import the GR4JCN model and write the configured files. Make sure the end date is before the end of the
# hydrometeorological data NetCDF file.
from ravenpy.new_config.emulators import GR4JCN
from ravenpy.new_config import commands as rc
from ravenpy import Emulator
from ravenpy.ravenpy import run

# Start and end date
start_date = dt.datetime(1985, 1, 1)
end_date = dt.datetime(1990, 1, 1)

# Define HRU
hru = {}
hru = dict(
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    hru_type="land",
)

# Set alt_names
alt_names = {
    "TEMP_MIN": "tmin",
    "TEMP_MAX": "tmax",
    "PRECIP": "pr",
}

# prepare data_type
data_type = ["TEMP_MAX", "TEMP_MIN", "PRECIP"]

#    Now that we have the single file containing tmax, tmin and pr, we can setup a single gauge that contains all three.
m = GR4JCN(
    params=[0.529, -3.396, 407.29, 1.072, 16.9, 0.947],
    Gauge=rc.Gauge.from_nc(
        "ERA5_weather_data.nc",
        data_type=data_type,  
        alt_names=alt_names,
        extra={
            1: {
                "elevation": hru["elevation"],
                "latitude": hru["latitude"],
                "longitude": hru["longitude"],
            }
        },
    ),
    HRUs=[hru],
    StartDate=start_date,
    EndDate=end_date,
    RunName="test_hotstart",
    GlobalParameter={"AVG_ANNUAL_RUNOFF": 208.480},
)

# Value for server, to change
# m = write_rv(workdir="/notebook_dir/writable-workspace/run_results_NB4", overwrite = True)
m.write_rv(workdir="/home/ets/src/run_results_NB7", overwrite=True)

# Prepare the emulator by writing files on disk
e = Emulator(config=m, workdir="/home/ets/src/run_results_NB7")

# Run the model and get the outputs.
outputs = e.run()

Now that we have run the model, we can extract the hydrograph:

In [ ]:
first_hydrograph = outputs.hydrograph.q_sim

### Now let's see if we can replicate the hydrograph using a hotstart file. 
First, run the model without a hotstart file, same as before, but only for one year:


In [ ]:
# Run the model, this time only with the first 3 years (1985-1988).
start_date = dt.datetime(1985, 1, 1)
end_date = dt.datetime(1988, 1, 1)

# Run the model
m = GR4JCN(
    params=[0.529, -3.396, 407.29, 1.072, 16.9, 0.947],
    Gauge=rc.Gauge.from_nc(
        "ERA5_weather_data.nc",
        data_type=data_type,  
        alt_names=alt_names,
        extra={
            1: {
                "elevation": hru["elevation"],
                "latitude": hru["latitude"],
                "longitude": hru["longitude"],
            }
        },
    ),
    HRUs=[hru],
    StartDate=start_date,
    EndDate=end_date,
    RunName="test_hotstart_a",
    GlobalParameter={"AVG_ANNUAL_RUNOFF": 208.480},
)

m.write_rv(workdir="/home/ets/src/run_results_NB7_a", overwrite=True)

# Prepare the emulator by writing files on disk
e = Emulator(config=m, workdir="/home/ets/src/run_results_NB7_a")

# Run the model and get the outputs.
outputs = e.run()


# Extract the hydrograph for this first year.
second_hydrograph_first_part = outputs.hydrograph.q_sim

# We can also keep the path to the solution (final model states) for later:
hotstart_states = outputs.files["solution"]

In [ ]:
outputs.files["solution"]


Now, let's run the model using the hotstart but for the final 2 years only (1988-1990). Notice that we have a new configuration variable called "rvc". This is where we can provide the path to the starting conditions!

In [ ]:
start_date = dt.datetime(1988, 1, 1)
end_date = dt.datetime(1990, 1, 1)

# Run the model
m = GR4JCN(
    params=[0.529, -3.396, 407.29, 1.072, 16.9, 0.947],
    Gauge=rc.Gauge.from_nc(
        "ERA5_weather_data.nc",
        data_type=data_type,  
        alt_names=alt_names,
        extra={
            1: {
                "elevation": hru["elevation"],
                "latitude": hru["latitude"],
                "longitude": hru["longitude"],
            }
        },
    ),
    HRUs=[hru],
    StartDate=start_date,
    EndDate=end_date,
    RunName="test_hotstart_b",
    GlobalParameter={"AVG_ANNUAL_RUNOFF": 208.480},
)

# Replace the default configuration of initial states with that of the final states of the previous run (hotstart)
m.set_solution(outputs.files["solution"])

# Write the config files
m.write_rv(workdir="/home/ets/src/run_results_NB7_b", overwrite=True)

# Prepare the emulator by writing files on disk
e = Emulator(config=m, workdir="/home/ets/src/run_results_NB7_b")

# Run the model and get the outputs.
outputs = e.run()

# Extract the hydrograph for this first year.
second_hydrograph_second_part = outputs.hydrograph.q_sim

Now that we have calculated both hydrographs separately, let's concatenate them into a single array of 2-years duration.

In [ ]:
second_hydrograph_complete = xr.concat(
    [second_hydrograph_first_part[0:-1], second_hydrograph_second_part], "time"
)

In [ ]:
first_hydrograph

In [ ]:
second_hydrograph_first_part

## Plotting both hydrographs
Let's plot both hydrographs to see if they are the same:

In [ ]:
second_hydrograph_second_part

In [ ]:
first_hydrograph.plot()
second_hydrograph_complete.plot()


And now if we look at the difference between both hydrographs:

In [ ]:
difference = first_hydrograph - second_hydrograph_complete
difference.plot()

We can see that differences are at machine precision levels, due to rounding in the hotstart file (note that the y-axis is 1e-6, which is essentially 0!). But the rest is perfect!

Therefore, we can provide forecasting abilities by saving simulation final states and using those to initialize model states for the forecasting runs. This will be used in other notebooks such as notebook #12 on hindcasting.
